In [1]:
import numpy as np

In [2]:
# Cubic (10 runs) mean fitness: 252
Cubic_pso_config_mean = [65.07982407, 4.95690714, 0.28949701, 0.54421663, 0.39890445, 1.42315492, 1.18854163]

# Linear (10 runs) mean fintess: 9.17486598344206e+24
Linear_pso_config_mean = [75.13534983, 6.59595661, 0.15486748, 0.63274028, 0.93549642, 1.5667033, 0.48415203]

# Tanh (10 runs) mean fitness: 23659 
Tanh_pso_config_mean = [54.98334367, 6.41212707, 0.59507185, 1.23273439, 1.68258425, 1.01222575, 0.19820747]

In [3]:
# Cubic Most fit model: 6847
Cubic_ann_params = np.array([ 4.06948631e+00, -7.29125756e-01,  5.73721712e-01, -4.97778569e-01, 1.21867021e-01, -4.07830431e-01,  9.23118308e-01,  9.96763804e-01, -3.53176479e-01,  2.82162360e+00, -6.77411971e-01, -6.33183382e-01, 3.17935306e-01, -1.91733120e-01,  2.57710600e-03])

#Linear Most fit model: 9.17486598344206e+25
Linear_ann_params = np.array([0.33653903, 0.62395429, 0.38036724,-0.73197691, 0.27957089, -0.44447831, 0.90125204, -0.72400064, -0.73325263, 0.2053941, -0.37270846, 0.1568939, 0.4157871, -0.37861424, -0.57400409])

#Sine Most fit model: 329
Sine_ann_params = np.array([ 3.50680037,  0.2952162 ,  0.71664711, -0.95665899, -0.9055195, -0.27101866,  0.41973435,  0.36889121, -0.28896977,  3.3595307, 0.42623047,  0.30804742,  0.62065839, -0.73504326,  0.70304275])

#Tanh Most fit model: 20498
Tanh_ann_params = np.array([ 4.30993349,  0.2291253 , -0.99626464, -0.29776704,  0.99032436, 0.54519926, 0.98178641, -0.06046016, 0.9675356 ,  2.17890815, -0.49269196, -0.29330431, -0.94045692, 0.72556016, 0.95398864])



In [4]:
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO import history as hs
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

data = d.Data("./Data/1in_linear.txt")
decimals = 'linear'
X = data.get_rows()
y = data.get_output()

psoParams = Linear_pso_config_mean
annParams = Linear_ann_params

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

D:\GitProjects\Bio-comp\Coursework\ANNModel\model.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = hs.PSOHistory(model)

Model Compiled!


In [6]:
pso = ps.PSO(swarm_size=100, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 400}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6)
pso = pso.decode_vec(psoParams)

print(pso.swarm_size)

75


### Run experiment with the PSO hyperparameters calculated by meta-PSO 
#### This will find the 10 most fit ANN parameters, produce a mean accuracy*, Fitness, and loss for both test and train.

*This accuracy is calculated with an absolute tolerance of 0.07 for all tests

In [29]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)

train ={
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

test = {
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

for i in range(10):
    fitness = pso.run()
    # Evaluating on training set
    model.set_training_input(X_train, y_train)
    train['fitnessList'].append(model.evaluate_fitness(pso.best.location))
    score = data.score(model, decimals, atol=0.07)
    train['scoreA'].append(score[0])
    train['scoreB'].append(score[1])
    train['lossList'].append(model.loss)
    # Evaluating on testing set
    model.set_training_input(X_test, y_test)
    test['fitnessList'].append(model.evaluate_fitness(pso.best.location))
    score = data.score(model, decimals, atol=0.07)
    test['scoreA'].append(score[0])
    test['scoreB'].append(score[1])
    test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))

Testing on train - A:  0.6029850746268657  B:  0.6029850746268657  Fitness:  312.487  Loss:  0.02
Testing on test - A:  0.6121212121212121  B:  0.6121212121212121  Fitness:  288.863  Loss:  0.018


In [8]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)
fitness = pso.run()
print(fitness.location)
print(fitness.fitness)

[ 3.35141629 -0.20317384 -0.38391576 -0.9161127  -0.06277832  0.28440381
  0.90208291  0.76177685  0.65481119  0.1802816  -0.58371951 -0.20222385
  0.59739635  0.97036189 -0.31231991]
28.68329476890625


In [9]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.NULL
bias:  [-0.58371951]
weights:  [[-0.20222385]
 [ 0.59739635]
 [ 0.97036189]
 [-0.31231991]]


In [28]:
model.decode_vec(annParams)
model.one_pass()

train ={
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

test = {
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

# Evaluating on training set
model.set_training_input(X_train, y_train)
train['fitnessList'].append(model.evaluate_fitness(annParams))
score = data.score(model, decimals, atol=0.07)
train['scoreA'].append(score[0])
train['scoreB'].append(score[1])
train['lossList'].append(model.loss)

# Evaluating on testing set
model.set_training_input(X_test, y_test)
test['fitnessList'].append(model.evaluate_fitness(annParams))
score = data.score(model, decimals, atol=0.07)
test['scoreA'].append(score[0])
test['scoreB'].append(score[1])
test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))


Testing on train - A:  1.0  B:  1.0  Fitness:  3.005049667997004e+17  Loss:  0.0
Testing on test - A:  1.0  B:  1.0  Fitness:  3.380238819540462e+17  Loss:  0.0


In [11]:
1/model.loss

3.380238819540462e+17

In [12]:
model.evaluate_fitness(Cubic_ann_params)

14.701404307863989